In [3]:
# %% [markdown]
# # Data Preprocessing for Credit Risk Model
# 
# ## Objective
# Prepare the raw data for modeling by handling missing values, encoding categorical variables,
# and creating a clean processed dataset.

# %% [markdown]
# ## 1. Setup and Load Data

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

# %% [markdown]

✅ Libraries imported successfully!


In [4]:
# ## 2. Load Raw Data Files

# %%
print("📂 Loading raw data files...")

# Load main transaction data
data = pd.read_csv("../data/raw/data.csv")
print(f"✅ Loaded data.csv: {data.shape}")

# Load variable definitions
var_defs = pd.read_csv("../data/raw/Xente_Variable_Definitions.csv")
print(f"✅ Loaded Xente_Variable_Definitions.csv: {var_defs.shape}")

print("\n🔍 Variable Definitions:")
display(var_defs.head(20))

print("\n🔍 Data sample (first 5 rows):")
display(data.head())

print("\n📊 Data info:")
print(f"Columns: {list(data.columns)}")
print(f"\nData types:\n{data.dtypes}")


📂 Loading raw data files...
✅ Loaded data.csv: (95662, 16)
✅ Loaded Xente_Variable_Definitions.csv: (16, 2)

🔍 Variable Definitions:


,Column Name,Definition
0,TransactionId,Unique �transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transacti...
2,AccountId,Unique number identifying the customer on plat...
3,SubscriptionId,Unique number identifying the customer subscri...
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item �bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader pr...



🔍 Data sample (first 5 rows):


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.000,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.000,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.000,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.000,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.000,644,2018-11-15T03:34:21Z,2,0



📊 Data info:
Columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'CountryCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId', 'Amount', 'Value', 'TransactionStartTime', 'PricingStrategy', 'FraudResult']

Data types:
TransactionId            object
BatchId                  object
AccountId                object
SubscriptionId           object
CustomerId               object
CurrencyCode             object
CountryCode               int64
ProviderId               object
ProductId                object
ProductCategory          object
ChannelId                object
Amount                  float64
Value                     int64
TransactionStartTime     object
PricingStrategy           int64
FraudResult               int64
dtype: object


In [5]:
# %% [markdown]
# ## 3. Explore Data Structure

# %%
print("🔍 Exploring data structure...")

# Basic statistics
print("\n📈 Basic statistics:")
display(data.describe())

# Check for missing values
print("\n🔍 Missing values analysis:")
missing_info = pd.DataFrame({
    'missing_count': data.isnull().sum(),
    'missing_percentage': (data.isnull().sum() / len(data)) * 100
}).sort_values('missing_percentage', ascending=False)

display(missing_info[missing_info['missing_count'] > 0])

print(f"\n📊 Total missing values: {data.isnull().sum().sum():,}")
print(f"Percentage of data missing: {(data.isnull().sum().sum() / (data.shape[0] * data.shape[1]) * 100):.2f}%")

# Visualize missing values
if missing_info['missing_count'].sum() > 0:
    plt.figure(figsize=(12, 6))
    missing_info[missing_info['missing_count'] > 0]['missing_percentage'].head(20).plot(
        kind='bar', color='coral'
    )
    plt.axhline(y=5, color='r', linestyle='--', alpha=0.5, label='5% threshold')
    plt.axhline(y=20, color='orange', linestyle='--', alpha=0.5, label='20% threshold')
    plt.title('Missing Values by Column (%)')
    plt.ylabel('Percentage Missing')
    plt.xlabel('Columns')
    plt.xticks(rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()

🔍 Exploring data structure...

📈 Basic statistics:


,CountryCode,Amount,Value,PricingStrategy,FraudResult
count,95662.000,95662.000,95662.000,95662.000,95662.000
mean,256.000,6717.846,9900.584,2.256,0.002
std,0.000,123306.797,123122.088,0.733,0.045
min,256.000,-1000000.000,2.000,0.000,0.000
25%,256.000,-50.000,275.000,2.000,0.000
50%,256.000,1000.000,1000.000,2.000,0.000
75%,256.000,2800.000,5000.000,2.000,0.000
max,256.000,9880000.000,9880000.000,4.000,1.000



🔍 Missing values analysis:


,missing_count,missing_percentage



📊 Total missing values: 0
Percentage of data missing: 0.00%


In [7]:
# %% [markdown]
# ## 4. Identify Key Columns

# %%
print("🔑 Identifying key columns...")

# Find important columns
customer_cols = [col for col in data.columns if any(x in col.lower() for x in ['cust', 'customer', 'client', 'id'])]
date_cols = [col for col in data.columns if any(x in col.lower() for x in ['date', 'time', 'timestamp'])]
amount_cols = [col for col in data.columns if any(x in col.lower() for x in ['amount', 'value', 'price', 'total', 'amt'])]
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()

print(f"📋 Customer ID columns: {customer_cols}")
print(f"📅 Date columns: {date_cols}")
print(f"💰 Amount columns: {amount_cols}")
print(f"🔤 Categorical columns: {categorical_cols[:10]}...")  # Show first 10

# Select primary columns
primary_customer_col = customer_cols[0] if customer_cols else data.columns[0]
primary_date_col = date_cols[0] if date_cols else None
primary_amount_col = amount_cols[0] if amount_cols else None

print(f"\n✅ Selected primary columns:")
print(f"  Customer ID: {primary_customer_col}")
print(f"  Date: {primary_date_col}")
print(f"  Amount: {primary_amount_col}")

🔑 Identifying key columns...
📋 Customer ID columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'ProviderId', 'ProductId', 'ChannelId']
📅 Date columns: ['TransactionStartTime']
💰 Amount columns: ['Amount', 'Value']
🔤 Categorical columns: ['TransactionId', 'BatchId', 'AccountId', 'SubscriptionId', 'CustomerId', 'CurrencyCode', 'ProviderId', 'ProductId', 'ProductCategory', 'ChannelId']...

✅ Selected primary columns:
  Customer ID: TransactionId
  Date: TransactionStartTime
  Amount: Amount


In [8]:
# %% [markdown]
# ## 5. Handle Missing Values

# %%
print("🔄 Handling missing values...")

# Create a copy for processing
processed_data = data.copy()
original_shape = processed_data.shape

print(f"Original shape: {original_shape}")

# Strategy for different types of columns
for col in processed_data.columns:
    missing_count = processed_data[col].isnull().sum()
    
    if missing_count > 0:
        col_type = processed_data[col].dtype
        
        if col_type == 'object':  # Categorical columns
            # Fill with mode or 'Unknown'
            if missing_count / len(processed_data) < 0.1:  # Less than 10% missing
                mode_val = processed_data[col].mode()[0] if not processed_data[col].mode().empty else 'Unknown'
                processed_data[col] = processed_data[col].fillna(mode_val)
                print(f"  {col}: Filled {missing_count} missing with mode '{mode_val}'")
            else:
                processed_data[col] = processed_data[col].fillna('Unknown')
                print(f"  {col}: Filled {missing_count} missing with 'Unknown'")
                
        elif np.issubdtype(col_type, np.number):  # Numerical columns
            # Fill with median
            median_val = processed_data[col].median()
            processed_data[col] = processed_data[col].fillna(median_val)
            print(f"  {col}: Filled {missing_count} missing with median {median_val:.2f}")
            
        elif 'date' in col.lower() or 'time' in col.lower():  # Date columns
            # Fill with most recent date or specific date
            if processed_data[col].notna().any():
                most_recent = processed_data[col].max()
                processed_data[col] = processed_data[col].fillna(most_recent)
                print(f"  {col}: Filled {missing_count} missing with most recent date")
            else:
                processed_data[col] = processed_data[col].fillna(pd.Timestamp('2023-01-01'))
                print(f"  {col}: Filled {missing_count} missing with default date")

print(f"\n✅ Missing values handled. Remaining missing: {processed_data.isnull().sum().sum()}")


🔄 Handling missing values...
Original shape: (95662, 16)

✅ Missing values handled. Remaining missing: 0


In [9]:
# %% [markdown]
# ## 6. Convert Data Types

# %%
print("🔄 Converting data types...")

# Convert date columns
if primary_date_col and primary_date_col in processed_data.columns:
    processed_data[primary_date_col] = pd.to_datetime(processed_data[primary_date_col], errors='coerce')
    print(f"✅ Converted {primary_date_col} to datetime")

# Convert amount columns to numeric
if primary_amount_col and primary_amount_col in processed_data.columns:
    processed_data[primary_amount_col] = pd.to_numeric(processed_data[primary_amount_col], errors='coerce')
    print(f"✅ Converted {primary_amount_col} to numeric")

# Check data types after conversion
print("\n📊 Data types after conversion:")
print(processed_data.dtypes.head(15))

🔄 Converting data types...
✅ Converted TransactionStartTime to datetime
✅ Converted Amount to numeric

📊 Data types after conversion:
TransactionId                        object
BatchId                              object
AccountId                            object
SubscriptionId                       object
CustomerId                           object
CurrencyCode                         object
CountryCode                           int64
ProviderId                           object
ProductId                            object
ProductCategory                      object
ChannelId                            object
Amount                              float64
Value                                 int64
TransactionStartTime    datetime64[ns, UTC]
PricingStrategy                       int64
dtype: object


In [10]:
# %% [markdown]
# ## 7. Create Customer-Level Features

# %%
print("🏗️ Creating customer-level features...")

if primary_customer_col in processed_data.columns:
    # Group by customer
    customer_features = []
    
    # Basic transaction statistics
    if primary_amount_col:
        amount_stats = processed_data.groupby(primary_customer_col)[primary_amount_col].agg([
            'count', 'sum', 'mean', 'std', 'min', 'max', 'median'
        ]).round(2)
        amount_stats.columns = [f'amount_{col}' for col in amount_stats.columns]
        customer_features.append(amount_stats)
    
    # Date-based features
    if primary_date_col:
        # Ensure it's datetime
        if processed_data[primary_date_col].dtype == 'datetime64[ns]':
            date_stats = processed_data.groupby(primary_customer_col)[primary_date_col].agg([
                'min', 'max', 'count'
            ])
            date_stats.columns = [f'date_{col}' for col in date_stats.columns]
            
            # Calculate recency (days since last transaction)
            latest_date = processed_data[primary_date_col].max()
            date_stats['days_since_last_transaction'] = (latest_date - date_stats['date_max']).dt.days
            date_stats['transaction_frequency_days'] = (date_stats['date_max'] - date_stats['date_min']).dt.days / date_stats['date_count']
            
            customer_features.append(date_stats)
    
    # Categorical feature aggregations (for key categorical columns)
    categorical_to_agg = [col for col in categorical_cols if col != primary_customer_col and processed_data[col].nunique() < 50]
    
    for cat_col in categorical_to_agg[:5]:  # Limit to first 5 to avoid explosion
        # Get most common category per customer
        most_common = processed_data.groupby([primary_customer_col, cat_col]).size().reset_index(name='count')
        idx = most_common.groupby(primary_customer_col)['count'].idxmax()
        most_common = most_common.loc[idx, [primary_customer_col, cat_col]]
        most_common.columns = [primary_customer_col, f'most_common_{cat_col}']
        most_common.set_index(primary_customer_col, inplace=True)
        customer_features.append(most_common)
    
    # Merge all customer features
    if customer_features:
        processed_customers = customer_features[0]
        for df in customer_features[1:]:
            processed_customers = processed_customers.join(df, how='outer')
        
        processed_customers = processed_customers.reset_index()
        print(f"✅ Created customer-level dataset: {processed_customers.shape}")
        
        # Fill any remaining NaN values
        processed_customers = processed_customers.fillna(0)
        
    else:
        # Fallback: simple count
        processed_customers = processed_data.groupby(primary_customer_col).size().reset_index(name='transaction_count')
        print(f"✅ Created simple customer dataset: {processed_customers.shape}")
    
    # Rename customer column for consistency
    processed_customers.rename(columns={primary_customer_col: 'CustomerId'}, inplace=True)
    
else:
    print("❌ Could not create customer features. Customer column not found.")
    processed_customers = pd.DataFrame()

print("\n🔍 Customer features sample:")
if not processed_customers.empty:
    display(processed_customers.head())
    print(f"\n📊 Customer features shape: {processed_customers.shape}")
    print(f"📊 Customer features columns: {list(processed_customers.columns)}")


🏗️ Creating customer-level features...
✅ Created customer-level dataset: (95662, 13)

🔍 Customer features sample:


,CustomerId,amount_count,amount_sum,amount_mean,amount_std,amount_min,amount_max,amount_median,most_common_CurrencyCode,most_common_ProviderId,most_common_ProductId,most_common_ProductCategory,most_common_ChannelId
0,TransactionId_1,1,5000.000,5000.000,0.000,5000.000,5000.000,5000.000,UGX,ProviderId_1,ProductId_15,financial_services,ChannelId_3
1,TransactionId_100,1,-100.000,-100.000,0.000,-100.000,-100.000,-100.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2
2,TransactionId_1000,1,-5000.000,-5000.000,0.000,-5000.000,-5000.000,-5000.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2
3,TransactionId_10000,1,500.000,500.000,0.000,500.000,500.000,500.000,UGX,ProviderId_6,ProductId_3,airtime,ChannelId_3
4,TransactionId_100001,1,-30.000,-30.000,0.000,-30.000,-30.000,-30.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2



📊 Customer features shape: (95662, 13)
📊 Customer features columns: ['CustomerId', 'amount_count', 'amount_sum', 'amount_mean', 'amount_std', 'amount_min', 'amount_max', 'amount_median', 'most_common_CurrencyCode', 'most_common_ProviderId', 'most_common_ProductId', 'most_common_ProductCategory', 'most_common_ChannelId']


In [11]:
# %% [markdown]
# ## 8. Handle Outliers

# %%
print("📊 Handling outliers...")

if not processed_customers.empty:
    # Identify numerical columns
    numerical_cols = processed_customers.select_dtypes(include=[np.number]).columns.tolist()
    
    # Remove ID columns
    numerical_cols = [col for col in numerical_cols if 'id' not in col.lower()]
    
    print(f"Numerical columns to check for outliers: {numerical_cols}")
    
    # Apply winsorization (cap outliers)
    for col in numerical_cols[:10]:  # Limit to first 10 columns
        if col in processed_customers.columns:
            q1 = processed_customers[col].quantile(0.01)
            q3 = processed_customers[col].quantile(0.99)
            
            # Count outliers
            outliers_before = ((processed_customers[col] < q1) | (processed_customers[col] > q3)).sum()
            
            if outliers_before > 0:
                # Cap outliers
                processed_customers[col] = processed_customers[col].clip(lower=q1, upper=q3)
                print(f"  {col}: Capped {outliers_before} outliers ({outliers_before/len(processed_customers)*100:.1f}%)")
    
    print("✅ Outliers handled using 1st-99th percentile winsorization")


📊 Handling outliers...
Numerical columns to check for outliers: ['amount_count', 'amount_sum', 'amount_mean', 'amount_std', 'amount_min', 'amount_max', 'amount_median']
  amount_sum: Capped 1395 outliers (1.5%)
  amount_mean: Capped 1395 outliers (1.5%)
  amount_min: Capped 1395 outliers (1.5%)
  amount_max: Capped 1395 outliers (1.5%)
  amount_median: Capped 1395 outliers (1.5%)
✅ Outliers handled using 1st-99th percentile winsorization


In [12]:
# %% [markdown]
# ## 9. Save Processed Data

# %%
print("💾 Saving processed data...")

import os

# Create processed directory if it doesn't exist
os.makedirs("../data/processed/", exist_ok=True)

if not processed_customers.empty:
    # Save customer-level processed data
    output_path = "../data/processed/processed_data.csv"
    processed_customers.to_csv(output_path, index=False)
    print(f"✅ Customer-level processed data saved to: {output_path}")
    print(f"   Shape: {processed_customers.shape}")
    print(f"   Columns: {len(processed_customers.columns)}")
    
    # Also save the transaction-level processed data
    transaction_output_path = "../data/processed/processed_transactions.csv"
    processed_data.to_csv(transaction_output_path, index=False)
    print(f"✅ Transaction-level processed data saved to: {transaction_output_path}")
    print(f"   Shape: {processed_data.shape}")
    
    # Save a summary file
    summary = pd.DataFrame({
        'dataset': ['customer_level', 'transaction_level'],
        'rows': [processed_customers.shape[0], processed_data.shape[0]],
        'columns': [processed_customers.shape[1], processed_data.shape[1]],
        'file_path': [output_path, transaction_output_path]
    })
    
    summary_path = "../data/processed/processing_summary.csv"
    summary.to_csv(summary_path, index=False)
    print(f"✅ Processing summary saved to: {summary_path}")
    
    # Display sample of saved data
    print("\n📋 Sample of saved customer data:")
    display(processed_customers.head())
    
else:
    print("❌ No processed data to save.")

# %% [markdown]
# ## 10. Summary

# %%
print("=" * 80)
print("✅ DATA PREPROCESSING COMPLETED")
print("=" * 80)

if not processed_customers.empty:
    print(f"\n📊 FINAL DATASET:")
    print(f"  Rows: {processed_customers.shape[0]:,} customers")
    print(f"  Columns: {processed_customers.shape[1]} features")
    print(f"  Saved to: data/processed/processed_data.csv")
    
    print(f"\n🔍 Key features created:")
    feature_types = {
        'Transaction counts': [col for col in processed_customers.columns if 'count' in col.lower()],
        'Amount statistics': [col for col in processed_customers.columns if 'amount' in col.lower()],
        'Date features': [col for col in processed_customers.columns if 'date' in col.lower() or 'days' in col.lower()],
        'Categorical features': [col for col in processed_customers.columns if 'most_common' in col.lower()]
    }
    
    for feature_type, features in feature_types.items():
        if features:
            print(f"  • {feature_type}: {len(features)} features")
            if len(features) <= 5:
                print(f"    {features}")
    
    print(f"\n📈 Data quality check:")
    print(f"  Missing values: {processed_customers.isnull().sum().sum()}")
    print(f"  Duplicate customers: {processed_customers.duplicated().sum()}")
    print(f"  Memory usage: {processed_customers.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n🎯 Next steps:")
print("  1. Run Task 4 (Target Engineering) to create 'is_high_risk' target")
print("  2. Use processed_data.csv for model training")
print("  3. Proceed with feature engineering and model development")

print("\n" + "=" * 80)

💾 Saving processed data...
✅ Customer-level processed data saved to: ../data/processed/processed_data.csv
   Shape: (95662, 13)
   Columns: 13
✅ Transaction-level processed data saved to: ../data/processed/processed_transactions.csv
   Shape: (95662, 16)
✅ Processing summary saved to: ../data/processed/processing_summary.csv

📋 Sample of saved customer data:


,CustomerId,amount_count,amount_sum,amount_mean,amount_std,amount_min,amount_max,amount_median,most_common_CurrencyCode,most_common_ProviderId,most_common_ProductId,most_common_ProductCategory,most_common_ChannelId
0,TransactionId_1,1,5000.000,5000.000,0.000,5000.000,5000.000,5000.000,UGX,ProviderId_1,ProductId_15,financial_services,ChannelId_3
1,TransactionId_100,1,-100.000,-100.000,0.000,-100.000,-100.000,-100.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2
2,TransactionId_1000,1,-5000.000,-5000.000,0.000,-5000.000,-5000.000,-5000.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2
3,TransactionId_10000,1,500.000,500.000,0.000,500.000,500.000,500.000,UGX,ProviderId_6,ProductId_3,airtime,ChannelId_3
4,TransactionId_100001,1,-30.000,-30.000,0.000,-30.000,-30.000,-30.000,UGX,ProviderId_4,ProductId_6,financial_services,ChannelId_2


✅ DATA PREPROCESSING COMPLETED

📊 FINAL DATASET:
  Rows: 95,662 customers
  Columns: 13 features
  Saved to: data/processed/processed_data.csv

🔍 Key features created:
  • Transaction counts: 1 features
    ['amount_count']
  • Amount statistics: 7 features
  • Categorical features: 5 features
    ['most_common_CurrencyCode', 'most_common_ProviderId', 'most_common_ProductId', 'most_common_ProductCategory', 'most_common_ChannelId']

📈 Data quality check:
  Missing values: 0
  Duplicate customers: 0
  Memory usage: 38.22 MB

🎯 Next steps:
  1. Run Task 4 (Target Engineering) to create 'is_high_risk' target
  2. Use processed_data.csv for model training
  3. Proceed with feature engineering and model development

